### 5. **Análisis de Correlación y Regresión**
**Resumen:** Se centra en la identificación de relaciones significativas entre las variables numéricas y el desarrollo de modelos de regresión para hacer predicciones basadas en esas relaciones.

In [ ]:
# Llegim les dades del dataframe y preparem la columna date correctament

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
df = pd.read_csv("avocado.csv")
df_cp = df.copy()
df_cp['Date'] = pd.to_datetime(df_cp['Date'])

region_classification = {
'Albany': 'City',
'Atlanta': 'City',
'BaltimoreWashington': 'Region',
'Boise': 'City',
'Boston': 'City',
'BuffaloRochester': 'Region',
'California': 'GreaterRegion',
'Charlotte': 'City',
'Chicago': 'City',
'CincinnatiDayton': 'Region',
'Columbus': 'City',
'DallasFtWorth': 'Region',
'Denver': 'City',
'Detroit': 'City',
'GrandRapids': 'City',
'GreatLakes': 'GreaterRegion',
'HarrisburgScranton': 'Region',
'HartfordSpringfield': 'Region',
'Houston': 'City',
'Indianapolis': 'City',
'Jacksonville': 'City',
'LasVegas': 'City',
'LosAngeles': 'City',
'Louisville': 'City',
'MiamiFtLauderdale': 'Region',
'Midsouth': 'GreaterRegion',
'Nashville': 'City',
'NewOrleansMobile': 'Region',
'NewYork': 'City',
'Northeast': 'GreaterRegion',
'NorthernNewEngland': 'Region',
'Orlando': 'City',
'Philadelphia': 'City',
'PhoenixTucson': 'Region',
'Pittsburgh': 'City',
'Plains': 'GreaterRegion',
'Portland': 'City',
'RaleighGreensboro': 'Region',
'RichmondNorfolk': 'Region',
'Roanoke': 'City',
'Sacramento': 'City',
'SanDiego': 'City',
'SanFrancisco': 'City',
'Seattle': 'City',
'SouthCarolina': 'Region',
'SouthCentral': 'GreaterRegion',
'Southeast': 'GreaterRegion',
'Spokane': 'City',
'StLouis': 'City',
'Syracuse': 'City',
'Tampa': 'City',
'TotalUS': 'TotalUS',
'West': 'GreaterRegion',
'WestTexNewMexico': 'Region'
}

classification_colors = {'City':'green' ,'Region':'yellow' ,'GreaterRegion':'orange', 'State':'red', 'TotalUS': 'blue'}

df_cp['region_class']= df_cp['region'].map(region_classification)

df_cp = df_cp.rename(columns={df.columns[0]: 'Col_0'}) # Primera columna sin titulo, potencialmente eliminable
df_cp = df_cp.rename(columns={'4046': 'Volume_Hass_S'}) # Etiquetas mas descritivas
df_cp = df_cp.rename(columns={'4225': 'Volume_Hass_L'})
df_cp = df_cp.rename(columns={'4770': 'Volume_Hass_XL'})
df_cp_cleaned = df_cp.drop('Col_0', axis=1) # Parecen IDs del 0 al 52. Eliminable. # Col_0 = df_cp['Col_0'].unique()  print(f"Col_0: {Col_0}\n")
df_cp_cleaned = df_cp[df_cp.region != 'TotalUS'] # Para seleccionar unicamente las regiones propias , descartamos Total US para la vista gráfica

## 1. **Matriz de Correlación:** 
   - **Uso de Datos:** Utiliza las columnas numéricas del DataFrame (p. ej., `AveragePrice`, `Total Volume`, `4046`, `4225`, `4770`, `Total Bags`).
   - **Esperado:** 
     - Importa las librerías necesarias: `import seaborn as sns` y `import matplotlib.pyplot as plt`.
     - Calcula la matriz de correlación usando el método `.corr()` del DataFrame.
     - Visualiza la matriz utilizando `sns.heatmap()`. 
     - Anota las correlaciones más significativas y discute su posible impacto en el análisis.

In [ ]:
df_corr_gen= df_cp.copy()

df_corr_gen = df_corr_gen[df_corr_gen['region']=='Philadelphia']

# df_corr_gen = df_corr_gen[(df_corr_gen['region_class']== 'City') | (df_corr_gen['region_class']== 'Region')]

corr_df= df_corr_gen[['AveragePrice', 'Total Volume', 'Total Bags' ,'Volume_Hass_S', 'Volume_Hass_L', 'Volume_Hass_XL']]
# Calcular la matriz de correlación
corr_matrix = corr_df.corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación')
plt.show()

# Identificar columnas con correlación alta (umbral = 0.8)
threshold = 0.8
to_drop = []
for column in corr_matrix.columns:
    if any((corr_matrix[column].abs() > threshold) & (corr_matrix.index != column)):
        to_drop.append(column)
        
print(f"Variables altamente correlacionadas con otras: {to_drop}")


## 2. **Análisis de Dispersión entre Variables Clave:** 
   - **Uso de Datos:** Selecciona variables numéricas de interés como `AveragePrice` y `Total Volume`.
   - **Esperado:** 
     - Importa las librerías necesarias: `import seaborn as sns` y `import matplotlib.pyplot as plt`.
     - Crea un gráfico de dispersión con `sns.scatterplot()` para visualizar la relación entre `AveragePrice` y `Total Volume`.
     - Añade una línea de regresión utilizando `sns.regplot()` para ilustrar las tendencias.
     - Compara el ajuste de una regresión lineal frente a una polinómica.

In [ ]:
df_local = df_cp.copy()
df_local= df_local[df_local['type']=='organic']
df_local= df_local[df_local['region']=='TotalUS']
df_local = df_local[df_local['AveragePrice']==1.0]
display(df_local)

In [ ]:
# df_local = df_cp.copy()
# df_local= df_local[df_local['type']=='conventional']

# # df_local = df_cp[(df_cp['region']!='TotalUS')&(df_cp['region_class']!='GreaterRegion')].copy()
# df_local_USA=df_local[df_local['region_class']=='TotalUS']

# fig, ax= plt.subplots(figsize=(10,6))

# # sns.scatterplot(data= df_local , x=df_local['Total Volume'], y=df_local['AveragePrice'], alpha= 0.1)
# sns.regplot(data= df_local_USA , x=df_local_USA['Total Volume'], y=df_local_USA['AveragePrice'], order=1)#, line=True)
# plt.xscale('log')
# plt.grid()
# plt.title('Conventional')
# plt.figure()

df_local = df_cp.copy()
df_local= df_local[df_local['type']=='organic']

# df_local = df_cp[(df_cp['region']!='TotalUS')&(df_cp['region_class']!='GreaterRegion')].copy()
df_local_USA=df_local[df_local['region_class']=='TotalUS']

fig, ax= plt.subplots(figsize=(10,6))

# sns.scatterplot(data= df_local , x=df_local['Total Volume'], y=df_local['AveragePrice'], alpha= 0.1)
sns.regplot(data= df_local_USA , x=df_local_USA['Total Volume'], y=df_local_USA['AveragePrice'], order=1)#, line=True)
plt.xscale('log')
plt.grid()
plt.title('TotalUS')
plt.figure()

In [ ]:
df_local = df_cp.copy()
df_local= df_local[df_local['type']=='organic']

# df_local = df_cp[(df_cp['region']!='TotalUS')&(df_cp['region_class']!='GreaterRegion')].copy()
df_local_USA=df_local[df_local['region_class']=='TotalUS']
df_local_USA=df_local_USA[df_local_USA['AveragePrice']>1.1]

fig, ax= plt.subplots(figsize=(10,6))

# sns.scatterplot(data= df_local , x=df_local['Total Volume'], y=df_local['AveragePrice'], alpha= 0.1)
sns.regplot(data= df_local_USA , x=df_local_USA['Total Volume'], y=df_local_USA['AveragePrice'], order=1)#, line=True)
plt.xscale('log')
plt.grid()
plt.title('TotalUS Organic')
plt.figure()

# df_local = df_cp[(df_cp['region']!='TotalUS')&(df_cp['region_class']!='GreaterRegion')].copy()
df_local_region=df_local[df_local['region_class']=='City']

fig, ax= plt.subplots(figsize=(10,6))

# sns.scatterplot(data= df_local , x=df_local['Total Volume'], y=df_local['AveragePrice'], alpha= 0.1)
sns.regplot(data= df_local_region , x=df_local_region['Total Volume'], y=df_local_region['AveragePrice'], order=1)#, line=True)
plt.xscale('log')
plt.title('City Organic')
plt.grid()
plt.figure()

In [ ]:
df_local = df_cp.copy()
df_local= df_local[df_local['type']=='organic']

# df_local = df_cp[(df_cp['region']!='TotalUS')&(df_cp['region_class']!='GreaterRegion')].copy()
df_local_USA=df_local[df_local['region_class']=='TotalUS']

fig, ax= plt.subplots(figsize=(10,6))

# sns.scatterplot(data= df_local , x=df_local['Total Volume'], y=df_local['AveragePrice'], alpha= 0.1)
sns.regplot(data= df_local_USA , x=df_local_USA['Total Volume'], y=df_local_USA['AveragePrice'], order=1)#, line=True)
plt.xscale('log')
plt.grid()
plt.title('TotalUS')
plt.figure()

# df_local = df_cp[(df_cp['region']!='TotalUS')&(df_cp['region_class']!='GreaterRegion')].copy()
df_local_region=df_local[df_local['region_class']=='City']

fig, ax= plt.subplots(figsize=(10,6))

# sns.scatterplot(data= df_local , x=df_local['Total Volume'], y=df_local['AveragePrice'], alpha= 0.1)
sns.regplot(data= df_local_region , x=df_local_region['Total Volume'], y=df_local_region['AveragePrice'], order=1)#, line=True)
plt.xscale('log')
plt.title('City')
plt.grid()
plt.figure()

3. **Predicciones Mensuales Usando Datos Trimestrales:**
   - **Uso de Datos:** Agrupa datos por trimestres y segmenta en meses utilizando `Date`, `AveragePrice`, y `Total Volume`.
   - **Esperado:** 
     - Convierte la columna `Date` a tipo datetime si es necesario.
     - Agrupa los datos por trimestre y calcula el promedio de `AveragePrice` y `Total Volume`.
     - Utiliza los datos de los primeros 2 meses de un trimestre para predecir el precio del tercer mes.
     - Compara los resultados de las predicciones con los precios reales.
     - Evalúa la precisión de tus predicciones utilizando métricas como R² y RMSE.


In [ ]:
df_local= df_cp.copy()

# df_local= df_local[df_local['region']=='West']
# df_local= df_local[df_local['type']=='conventional']

df_local['Month'] = df_local['Date'].dt.to_period('M')
df_local['Day'] = df_local['Date'].dt.to_period('d')#.weekday()
df_local['Quarter'] = df_local['Date'].dt.to_period('Q')
df_local['Week'] = df_local['Date'].dt.to_period('W')
df_local['Quarter Month'] = df_local['Month'].map(lambda x: (x.month - 1) % 3 + 1 )

df_fit = df_local[df_local['Quarter Month']!=3 ]
df_check = df_local[df_local['Quarter Month']==3 ]

group_fit = df_fit.groupby('Quarter').agg({'AveragePrice':'mean', 'Total Volume':'mean' }).reset_index()
group_fit['Quarter_n']=group_fit['Quarter'].map(lambda date: (date.month - 1) // 3 + 1)
# group_fit['Total Volume'] = np.log(group_fit['Total Volume'])

group_check = df_check.groupby('Quarter').agg({'AveragePrice':'mean', 'Total Volume':'mean' }).reset_index()
group_check['Quarter_n']=group_fit['Quarter'].map(lambda date: (date.month - 1) // 3 + 1)
# group_check['Total Volume'] = np.log(group_check['Total Volume'])

# group_fit.info()

# df_local

# group_fit


In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
# Importar las bibliotecas necesarias para el modelo
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Dividir los datos en conjuntos de entrenamiento y prueba
# group_fit = df_fit.groupby('Quarter').agg({'AveragePrice':'mean', 'Total Volume':'mean' }).reset_index()
X_train= group_fit['Total Volume'].to_frame()
y_train= group_fit['AveragePrice']

# group_check = df_fit.groupby('Quarter').agg({'AveragePrice':'mean', 'Total Volume':'mean' }).reset_index()
X_test= group_check['Total Volume'].to_frame()
y_test= group_check['AveragePrice']

# ##########################################
# np.random.seed(42)
# X1 = np.random.rand(100)
# data = pd.DataFrame({
#     'X1': X1,
#     'X2': (X1 * 0.4) + 0.2,  # Alta correlación con X1
#     'X3': np.random.rand(100),               # Variable independiente
#     'X4': np.random.rand(100),               # Variable independiente
#     'Y': np.random.rand(100)                 # Variable objetivo
# })

# data_cleaned = data.drop(columns=['X2'])

# X = data_cleaned.drop(columns='Y')
# y = data_cleaned['Y']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# print(X_train)
##########################################
# Aplicar un modelo de regresión lineal para interpolar
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir los valores en el conjunto de prueba
y_pred = model.predict(X_test)

# Comparar los primeros 10 valores entre el valor predicho y el valor real
comparison_df = pd.DataFrame({
    'Valor Real': y_test.head(10).values,
    'Valor Predicho': y_pred[:10]
})

# Calcular el error porcentual
comparison_df['Error Porcentual'] = ((comparison_df['Valor Real'] - comparison_df['Valor Predicho']) / comparison_df['Valor Real']) * 100

print("\nComparación de los primeros 10 valores entre el valor predicho y el valor real:\n", comparison_df)


# Calcular el error cuadrático medio (MSE) para evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
print(f"\nError cuadrático medio (MSE) del modelo: {mse:.4f}")

# Calcular el valor R²
r2 = r2_score(y_test, y_pred)
print(f"\nValor R² del modelo: {r2:.4f}")

## 4. **Predicciones Trimestrales:**
   - **Uso de Datos:** Agrupa los datos en trimestres usando solo variables numéricas.
   - **Esperado:** 
     - Agrupa los datos por trimestres usando `pd.Grouper()` con `freq='Q'` para obtener promedios.
     - Usa los datos de 1 o 2 trimestres anteriores para predecir el siguiente trimestre ajustando modelos de regresión lineal y polinómica.
     - Compara los resultados de las predicciones con los precios reales.
     - Evalúa la precisión de tus predicciones utilizando métricas como R² y RMSE.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import avocado_manager as av
av.init()
df_cp = av.df("df_cp")
print(df_cp.info())

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error


df_local= df_cp.copy()

df_local= df_local[df_local['region']=='West']
df_local= df_local[df_local['type']=='conventional']

df_local['Month'] = df_local['Date'].dt.to_period('M')
#df_local['Day'] = df_local['Date'].dt.to_period('d')#.weekday()
df_local['Quarter'] = df_local['Date'].dt.to_period('Q')
df_local['Week'] = df_local['Date'].dt.to_period('W')
df_local['Quarter Month'] = df_local['Month'].map(lambda x: (x.month - 1) % 3 + 1 )

df_local['n_Quarter'] = df_local['Month'].map(lambda date: (date.month - 1) // 3 + 1)
display(df_local[['Month','Quarter','Week','Quarter Month']])

df_year = df_local[df_local['year']==2016]



########
# !!!!!!!
## TODO: Xavi random de l'ordre de les dades de train
# !!!!!!!

df_train = df_year[df_year['n_Quarter'].isin([1,2,3]) ]
df_test= df_year[df_year['n_Quarter']==4 ]
# display(df_local)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train= df_train['Total Volume'].to_frame()
y_train= df_train['AveragePrice']

X_test= df_test['Total Volume'].to_frame()
y_test= df_test['AveragePrice']

###########



# Aplicar un modelo de regresión lineal para interpolar
poly_features = PolynomialFeatures(degree=2)
poly_model = LinearRegression()

X_poly = poly_features.fit_transform(X_train)
poly_model.fit(X_poly, y_train)
y_pred_poly_train = poly_model.predict(X_poly)

X_test_poly= poly_features.fit_transform(X_test)
y_pred_poly_test = poly_model.predict(X_test_poly)

r2_poly_test = r2_score(y_test, y_pred_poly_test)
rmse_poly_test = root_mean_squared_error(y_test, y_pred_poly_test)
r2_poly_train = r2_score(y_train, y_pred_poly_train)
rmse_poly_train = root_mean_squared_error(y_train, y_pred_poly_train)

#     "coeficientes": poly_model.coef_}


# Comparar los primeros 10 valores entre el valor predicho y el valor real de train
comparison_df_train = pd.DataFrame({
    'Valor Real': y_test.head(10).values,
    'Valor Predicho': y_pred_poly_train[0:10]
})

# Comparar los primeros 10 valores entre el valor predicho y el valor real de test
comparison_df_tst = pd.DataFrame({
    'Valor Real': y_test.head(10).values,
    'Valor Predicho': y_pred_poly_test[0:10]
})


# Calcular el error porcentual
comparison_df_train['Error Porcentual'] = ((comparison_df_train['Valor Real'] - comparison_df_train['Valor Predicho']) / comparison_df_train['Valor Real']) * 100
print("\nComparación de los primeros 10 valores entre el valor predicho y el valor real de train:\n", comparison_df_train)

comparison_df_tst['Error Porcentual'] = ((comparison_df_tst['Valor Real'] - comparison_df_tst['Valor Predicho']) / comparison_df_tst['Valor Real']) * 100
print("\nComparación de los primeros 10 valores entre el valor predicho y el valor real de test:\n", comparison_df_tst)


# Calcular el error cuadrático medio (MSE) para evaluar el modelo
print(f"\nError cuadrático medio (MSE) del modelo train : {rmse_poly_train:.6f}")
# Calcular el valor R²
print(f"Valor R² del modelo train: {r2_poly_train:.6f}")

# Calcular el error cuadrático medio (MSE) para evaluar el modelo
print(f"\nError cuadrático medio (MSE) del modelo test: {rmse_poly_test:.6f}")
# Calcular el valor R²
print(f"Valor R² del modelo test: {r2_poly_test:.6f}")

## 5. **Predicciones Anuales:**
   - **Uso de Datos:** Agrupa los datos en años, utilizando únicamente columnas numéricas.
   - **Esperado:** 
     - Agrupa los datos por año utilizando `pd.Grouper()` con `freq='Y'`.
     - Usa los datos de 1 o 2 años anteriores para predecir el siguiente año ajustando modelos de regresión lineal y polinómica.
     - Evalúa la precisión de tus predicciones utilizando métricas como R² y RMSE.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error

df_local= df_cp.copy()

df_local= df_local[df_local['region']=='West']
# df_local= df_local[df_local['type']=='conventional']

df_local['Month'] = df_local['Date'].dt.to_period('M')
df_local['Day'] = df_local['Date'].dt.to_period('d')#.weekday()
df_local['Quarter'] = df_local['Date'].dt.to_period('Q')
df_local['Week'] = df_local['Date'].dt.to_period('W')
df_local['Quarter Month'] = df_local['Month'].map(lambda x: (x.month - 1) % 3 + 1 )
df_local['n_Quarter'] = df_local['Month'].map(lambda date: (date.month - 1) // 3 + 1)

# df_year = df_local[df_local['year']==2016]

########
df_train = df_local[df_local['year'].isin([2015,2016]) ]
df_test= df_local[df_local['year']==2017 ]
# display(df_train)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train= df_train['Total Volume'].to_frame()
y_train= df_train['AveragePrice']

X_test= df_test['Total Volume'].to_frame()
y_test= df_test['AveragePrice']

###########
# Aplicar un modelo de regresión lineal para interpolar
poly_features = PolynomialFeatures(degree=1)
poly_model = LinearRegression()

X_poly = poly_features.fit_transform(X_train)
poly_model.fit(X_poly, y_train)
y_pred_poly_train = poly_model.predict(X_poly)

X_test_poly= poly_features.fit_transform(X_test)
y_pred_poly_test = poly_model.predict(X_test_poly)


r2_poly_train = r2_score(y_test, y_pred_poly_test)
rmse_poly_train = mean_squared_error(y_test, y_pred_poly_test)

#     "coeficientes": poly_model.coef_}


# Comparar los primeros 10 valores entre el valor predicho y el valor real
comparison_df = pd.DataFrame({
    'Valor Real': y_test.head(10).values,
    'Valor Predicho': y_pred_poly_test[:10]
})

# Calcular el error porcentual
comparison_df['Error Porcentual'] = ((comparison_df['Valor Real'] - comparison_df['Valor Predicho']) / comparison_df['Valor Real']) * 100
print("\nComparación de los primeros 10 valores entre el valor predicho y el valor real:\n", comparison_df)

# Calcular el error cuadrático medio (MSE) para evaluar el modelo
print(f"\nError cuadrático medio (MSE) del modelo: {rmse_poly_train:.4f}")

# Calcular el valor R²
print(f"\nValor R² del modelo: {r2_poly_train:.4f}")


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import avocado_manager as av
av.init()
df_cp = av.df("df_cp")
print(df_cp.info())

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error


df_local= df_cp.copy()

df_local= df_local[df_local['region']=='West']
#df_local= df_local[df_local['type']=='conventional']

df_local['Month'] = df_local['Date'].dt.to_period('M')
#df_local['Day'] = df_local['Date'].dt.to_period('d')#.weekday()
df_local['Quarter'] = df_local['Date'].dt.to_period('Q')
df_local['Week'] = df_local['Date'].dt.to_period('W')
df_local['Quarter Month'] = df_local['Month'].map(lambda x: (x.month - 1) % 3 + 1 )

df_local['n_Quarter'] = df_local['Month'].map(lambda date: (date.month - 1) // 3 + 1)
display(df_local[['Month','Quarter','Week','Quarter Month']])

# df_year = df_local[df_local['year']==2016]


########
# !!!!!!!
## TODO: Xavi random de l'ordre de les dades de train
# !!!!!!!

df_train = df_local[df_local['year'].isin([2015,2016]) ]
df_test= df_local[df_local['year']==2017 ]
# display(df_local)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train= df_train['Total Volume'].to_frame()
y_train= df_train['AveragePrice']

X_test= df_test['Total Volume'].to_frame()
y_test= df_test['AveragePrice']

###########



# Aplicar un modelo de regresión lineal para interpolar
poly_features = PolynomialFeatures(degree=1)
poly_model = LinearRegression()

X_poly = poly_features.fit_transform(X_train)
poly_model.fit(X_poly, y_train)
y_pred_poly_train = poly_model.predict(X_poly)

X_test_poly= poly_features.fit_transform(X_test)
y_pred_poly_test = poly_model.predict(X_test_poly)

r2_poly_test = r2_score(y_test, y_pred_poly_test)
rmse_poly_test = root_mean_squared_error(y_test, y_pred_poly_test)
r2_poly_train = r2_score(y_train, y_pred_poly_train)
rmse_poly_train = root_mean_squared_error(y_train, y_pred_poly_train)

#     "coeficientes": poly_model.coef_}


# Comparar los primeros 10 valores entre el valor predicho y el valor real de train
comparison_df_train = pd.DataFrame({
    'Valor Real': y_test.head(10).values,
    'Valor Predicho': y_pred_poly_train[0:10]
})

# Comparar los primeros 10 valores entre el valor predicho y el valor real de test
comparison_df_tst = pd.DataFrame({
    'Valor Real': y_test.head(10).values,
    'Valor Predicho': y_pred_poly_test[0:10]
})


# Calcular el error porcentual
comparison_df_train['Error Porcentual'] = ((comparison_df_train['Valor Real'] - comparison_df_train['Valor Predicho']) / comparison_df_train['Valor Real']) * 100
print("\nComparación de los primeros 10 valores entre el valor predicho y el valor real de train:\n", comparison_df_train)

comparison_df_tst['Error Porcentual'] = ((comparison_df_tst['Valor Real'] - comparison_df_tst['Valor Predicho']) / comparison_df_tst['Valor Real']) * 100
print("\nComparación de los primeros 10 valores entre el valor predicho y el valor real de test:\n", comparison_df_tst)


# Calcular el error cuadrático medio (MSE) para evaluar el modelo
print(f"\nError cuadrático medio (MSE) del modelo train : {rmse_poly_train:.6f}")
# Calcular el valor R²
print(f"Valor R² del modelo train: {r2_poly_train:.6f}")

# Calcular el error cuadrático medio (MSE) para evaluar el modelo
print(f"\nError cuadrático medio (MSE) del modelo test: {rmse_poly_test:.6f}")
# Calcular el valor R²
print(f"Valor R² del modelo test: {r2_poly_test:.6f}")

## 6. **Desarrollo de Modelos de Regresión Múltiple:** 
   - **Uso de Datos:** Selecciona varias variables numéricas como `Total Volume`, `4046`, `4225`, `4770`, y `Total Bags` para predecir `AveragePrice`.
   - **Esperado:** 
     - Define las variables independientes (X) y dependientes (y).
     - Ajusta modelos de regresión múltiple.
     - Compara su rendimiento utilizando métricas como R² y RMSE y discute las implicaciones de los resultados.

In [ ]:
from sklearn import preprocessing

df_local = df_cp.copy()

df_local=df_local[df_local['region']=='West']
# df_local=df_local[df_local['type']=='conventional']

bool_org= {'organic':1, 'conventional':0}

df_local['Month'] = df_local['Date'].dt.to_period('M')
df_local['Quarter'] = df_local['Date'].dt.to_period('Q')
df_local['Quarter Month'] = df_local['Month'].map(lambda x: (x.month - 1) % 3 + 1 )
df_local['n_Quarter'] = df_local['Month'].map(lambda date: (date.month - 1) // 3 + 1)
df_local['type_Bool'] = df_local['type'].map(bool_org)

display(df_local)

In [ ]:
from sklearn import preprocessing

df_local = df_cp.copy()

df_local=df_local[df_local['region']=='West']
# df_local=df_local[df_local['type']=='conventional']

bool_org= {'organic':1, 'conventional':0}

df_local['Month'] = df_local['Date'].dt.to_period('M')
df_local['Quarter'] = df_local['Date'].dt.to_period('Q')
df_local['Quarter Month'] = df_local['Month'].map(lambda x: (x.month - 1) % 3 + 1 )
df_local['n_Quarter'] = df_local['Month'].map(lambda date: (date.month - 1) // 3 + 1)
df_local['type_Bool'] = df_local['type'].map(bool_org)

# Estandaritzar
df_local['Standard Volume'] = (df_local['Total Volume'] - df_local['Total Volume'].mean()) /df_local['Total Volume'].std()
df_local['ST_Volume_Hass_S'] = (df_local['Volume_Hass_S'] - df_local['Volume_Hass_S'].mean()) /df_local['Volume_Hass_S'].std()
df_local['ST_Volume_Hass_L'] = (df_local['Volume_Hass_L'] - df_local['Volume_Hass_L'].mean()) /df_local['Volume_Hass_L'].std()
df_local['ST_Volume_Hass_XL'] = (df_local['Volume_Hass_XL'] - df_local['Volume_Hass_XL'].mean()) /df_local['Volume_Hass_XL'].std()
df_local['Standard Bags'] = (df_local['Total Bags'] - df_local['Total Bags'].mean()) /df_local['Total Bags'].std()
# display(df_local)

#### versio polinomica Victor i Cesc per poder veure la diferencia amb la polinomica
# Aplicar un modelo de regresión lineal para interpolar
#poly_features = PolynomialFeatures(degree=1)
#X_poly = poly_features.fit_transform(X_train)

#poly_model = LinearRegression()
#poly_model.fit(X_poly, y_train)
# Predecir los valores en el conjunto de prueba
#y_pred_poly_train = poly_model.predict(X_poly)

#X_test_poly= poly_features.fit_transform(X_test)
#y_pred_poly_test = poly_model.predict(X_test_poly)
#### versio polinomica


Variables = ['Standard Volume', 'ST_Volume_Hass_S', 'ST_Volume_Hass_L', 'ST_Volume_Hass_XL', 'Standard Bags', 'n_Quarter', 'year', 'type_Bool']
# group_fit = df_fit.groupby('Quarter').agg({'AveragePrice':'mean', 'Total Volume':'mean' }).reset_index()
X= df_local[Variables]
Y= df_local['AveragePrice']#.to_frame()

X_train, X_test, y_train, y_test =  train_test_split(X, Y, test_size=0.2, random_state=42)

# group_check = df_fit.groupby('Quarter').agg({'AveragePrice':'mean', 'Total Volume':'mean' }).reset_index()
# X_test= df_year.loc[2:3,'Total Volume'].to_frame()
# y_test= df_year.loc[2:3,'AveragePrice']#.to_frame()

model = LinearRegression()
model.fit(X_train, y_train)
# Predecir los valores en el conjunto de prueba
y_pred = model.predict(X_test)

# Comparar los primeros 10 valores entre el valor predicho y el valor real
comparison_df = pd.DataFrame({
    'Valor Real': y_test.head(10).values,
    'Valor Predicho': y_pred[:10]
})

##  TODO: Xavi: Aixo es AveragePrice, correcte ? i no esta estandaritzat, correcte ?
display(y_test)

# Calcular el error porcentual
comparison_df['Error Porcentual'] = ((comparison_df['Valor Real'] - comparison_df['Valor Predicho']) / comparison_df['Valor Real']) * 100

print("\nComparación de los primeros 10 valores entre el valor predicho y el valor real:\n", comparison_df)
# Calcular el error cuadrático medio (MSE) para evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
print(f"\nError cuadrático medio (MSE) del modelo: {mse:.4f}")
for i in range(0, len(Variables)):    
    print(f'El coeficiente {Variables[i]}\t \t: {model.coef_[i]}')

# Calcular el valor R²
r2 = r2_score(y_test, y_pred)
print(f"\nValor R² del modelo: {r2:.4f}")

7. **Análisis de Coeficientes de Regresión Múltiple:**
   - **Uso de Datos:** Examina los coeficientes de los modelos de regresión múltiple ajustados.
   - **Esperado:** 
     - Extrae los coeficientes del modelo ajustado.
     - Interpreta los coeficientes para entender el impacto de cada variable numérica en `AveragePrice`.
     - Comenta sobre las variables más significativas y su relevancia.

In [ ]:
for i in range(0, len(Variables)):
    
    print(f'El coeficiente {Variables[i]}\t \t: {model.coef_[i]}')

8. **Modelos de Regresión para Diferenciar Volúmenes de Ventas:**
   - **Uso de Datos:** Usa `AveragePrice`, `Total Volume`, `4046`, `4225`, y `4770`.
   - **Esperado:** 
     - Ajusta modelos de regresión para analizar cómo los diferentes volúmenes de ventas afectan `AveragePrice`.
     - Compara los resultados de regresión lineal y polinómica.
     - Presenta las conclusiones de tus análisis.

In [ ]:
from sklearn import preprocessing

df_local = df_cp.copy()

df_local=df_local[df_local['region']=='West']
# df_local=df_local[df_local['type']=='conventional']

bool_org= {'organic':1, 'conventional':0}

df_local['Month'] = df_local['Date'].dt.to_period('M')
df_local['Quarter'] = df_local['Date'].dt.to_period('Q')
df_local['Quarter Month'] = df_local['Month'].map(lambda x: (x.month - 1) % 3 + 1 )
df_local['n_Quarter'] = df_local['Month'].map(lambda date: (date.month - 1) // 3 + 1)
df_local['type_Bool'] = df_local['type'].map(bool_org)

df_local['Standard Volume'] = (df_local['Total Volume'] - df_local['Total Volume'].mean()) /df_local['Total Volume'].std()
df_local['ST_Volume_Hass_S'] = (df_local['Volume_Hass_S'] - df_local['Volume_Hass_S'].mean()) /df_local['Volume_Hass_S'].std()
df_local['ST_Volume_Hass_L'] = (df_local['Volume_Hass_L'] - df_local['Volume_Hass_L'].mean()) /df_local['Volume_Hass_L'].std()
df_local['ST_Volume_Hass_XL'] = (df_local['Volume_Hass_XL'] - df_local['Volume_Hass_XL'].mean()) /df_local['Volume_Hass_XL'].std()
df_local['Standard Bags'] = (df_local['Total Bags'] - df_local['Total Bags'].mean()) /df_local['Total Bags'].std()


# display(df_local)

Variables = ['Standard Volume', 'ST_Volume_Hass_S', 'ST_Volume_Hass_L', 'ST_Volume_Hass_XL', 'Standard Bags', 'n_Quarter', 'year', 'type_Bool']
# group_fit = df_fit.groupby('Quarter').agg({'AveragePrice':'mean', 'Total Volume':'mean' }).reset_index()
X= df_local[Variables]
Y= df_local['AveragePrice']#.to_frame()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

###########
# Aplicar un modelo de regresión lineal para interpolar
poly_features = PolynomialFeatures(degree=2)
poly_model = LinearRegression()

X_poly = poly_features.fit_transform(X_train)
poly_model.fit(X_poly, y_train)
y_pred_poly_train = poly_model.predict(X_poly)

X_test_poly= poly_features.fit_transform(X_test)
y_pred_poly_test = poly_model.predict(X_test_poly)

# Comparar los primeros 10 valores entre el valor predicho y el valor real
comparison_df = pd.DataFrame({
    'Valor Real': y_test.head(10).values,
    'Valor Predicho': y_pred_poly_test[:10]
})

# Calcular el error porcentual
comparison_df['Error Porcentual'] = ((comparison_df['Valor Real'] - comparison_df['Valor Predicho']) / comparison_df['Valor Real']) * 100

print("\nComparación de los primeros 10 valores entre el valor predicho y el valor real:\n", comparison_df)


# Calcular el error cuadrático medio (MSE) para evaluar el modelo
mse = mean_squared_error(y_test, y_pred_poly_test)
print(f"\nError cuadrático medio (MSE) del modelo: {mse:.4f}")


for i in range(0, len(Variables)):
    
    print(f'El coeficiente {Variables[i]}\t \t: {model.coef_[i]}')

# Calcular el valor R²
r2 = r2_score(y_test, y_pred_poly_test)
print(f"\nValor R² del modelo: {r2:.4f}")

9. **Análisis de la Influencia de las Ventas Totales en el Precio Promedio:**
   - **Uso de Datos:** Usa `Total Volume`, `AveragePrice`, y `Total Bags`.
   - **Esperado:** 
     - Ajusta un modelo de regresión lineal y polinómica para ver cómo varía `AveragePrice` en función del volumen total de ventas.
     - Evalúa la significancia de los coeficientes y discute su relevancia.

10. **Regresión para Predecir el Precio Promedio Según el Volumen de Aguacates por Tipo:**
    - **Uso de Datos:** Usa `AveragePrice`, `4046`, `4225`, `4770`, y `Total Volume`.
    - **Esperado:** 
      - Ajusta modelos de regresión lineal y polinómica.
      - Evalúa la efectividad de ambos modelos utilizando métricas como R² y RMSE.
      - Discute cuál modelo ofrece mejores predicciones y por qué, basándote en los resultados obtenidos.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
df = pd.read_csv("avocado.csv")
df_cp = df.copy()
df_cp['Date'] = pd.to_datetime(df_cp['Date'])